In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
!pip install lightgbm==2.3.1
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from scipy.sparse import csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, vstack

import joblib as jb

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 1.2MB 2.8MB/s 
  Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Projetos/Rec_carros/clean_data.csv")
df.set_index('id', inplace= True)
df.head()

,modelo,y,ano,valor,cor,km,opcionais,link
id,,,,,,,,
1014060,c3 hatch attraction 1 2 12v,1,17/18,43800.0,Branca,38000.0,AirBagDuplo Alarme ArCondicionado ArDigital Câ...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1003530,c3 hatch attraction 1 2 12v,1,18/19,39500.0,Marrom,24536.0,AirBagDuplo Alarme ArCondicionado ArQuente Com...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1002771,c3 hatch attraction 1 5,0,15/15,29900.0,Branca,47000.0,AirBag AirBagDuplo Alarme ArCondicionado ArQue...,https://www.shopcar.com.br/veiculos/citro-n/c3...
981268,c3 hatch attraction 1 5,0,14/15,29900.0,Branca,0.0,AirBag Alarme ArCondicionado ArQuente Computad...,https://www.shopcar.com.br/veiculos/citro-n/c3...
1002011,c3 hatch attraction 1 6 16v,0,19/20,62990.0,Prata,0.0,AirBagDuplo ArCondicionado ArDigital ArQuente ...,https://www.shopcar.com.br/veiculos/citro-n/c3...


In [ ]:
#Separando Features
X_n= df[["valor", "km"]]
y= df['y']

X_l = df[['modelo', 'ano']]

#Random Forest

In [53]:
#Vetorizador
vectorizer_ano = TfidfVectorizer(min_df=1)
vectorizer_modelo = TfidfVectorizer(min_df=1)

ano_bow_train = vectorizer_ano.fit_transform(X_l['ano'])
modelo_bow_train = vectorizer_modelo.fit_transform(X_l['modelo'])

X_wbows = hstack([X_n, modelo_bow_train, ano_bow_train])

#Modelo
mdl_rf = RandomForestClassifier(n_estimators=700, random_state=0, 
                               max_depth=15,
                               class_weight="balanced", n_jobs=-1)  
mdl_rf.fit(X_wbows, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

#LGBM

In [ ]:
#Modelo
mdl_lgbm = LGBMClassifier(learning_rate=0.0066, num_leaves=18, max_depth=9, 
                      min_child_samples=1, subsample=0.936,
                      colsample_bytree=0.86, bagging_freq=1, n_estimators=500, 
                      random_state=0, class_weight="balanced", n_jobs=-1)
mdl_lgbm.fit(X_wbows, y)

LGBMClassifier(bagging_freq=1, boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=0.86, importance_type='split',
               learning_rate=0.0066, max_depth=9, min_child_samples=1,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=500,
               n_jobs=-1, num_leaves=18, objective=None, random_state=0,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.936,
               subsample_for_bin=200000, subsample_freq=0)

#Logistic Regression

In [ ]:
#Scaling e Modeling
X_wbows2 = csr_matrix(X_wbows.copy())

lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(n_jobs=-1, random_state=0, C= 6, solver= "liblinear"))
lr_pipeline.fit(X_wbows2, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Pipeline(memory=None,
         steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                ('logisticregression',
                 LogisticRegression(C=6, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=-1, penalty='l2',
                                    random_state=0, solver='liblinear',
                                    tol=0.0001, verbose=0, warm_start=False))],
         verbose=False)

#Salvando Modelos

In [54]:
jb.dump(mdl_lgbm, "/content/drive/My Drive/Projetos/Rec_carros/Modelos/lgbm_20082020.pkl.z")
jb.dump(mdl_rf, "/content/drive/My Drive/Projetos/Rec_carros/Modelos/random_forest_20082020.pkl.z")
jb.dump(lr_pipeline, "/content/drive/My Drive/Projetos/Rec_carros/Modelos/logistic_reg_20082020.pkl.z")
jb.dump(vectorizer_ano, "/content/drive/My Drive/Projetos/Rec_carros/Modelos/vectorizer_ano_20082020.pkl.z")
jb.dump(vectorizer_modelo, "/content/drive/My Drive/Projetos/Rec_carros/Modelos/vectorizer_modelo_20082020.pkl.z")

['/content/drive/My Drive/Projetos/Rec_carros/Modelos/vectorizer_modelo_20082020.pkl.z']